# Image Inpainting using CNNs
By Daniel Manser and Daniel Rotter


## Datensatz

## Problemstellung (Rotter)

## Herausforderungen (Rotter)

## Architektur (Manser)


## Quellen  
https://www.kaggle.com/code/vidhikishorwaghela/image-inpainting-
using-cnn

https://wandb.ai/wandb_fc/articles/reports/Introduction-to-image-
inpainting-with-deep-learning--Vmlldzo1NDI3MjA5